In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import BallTree
import hnswlib

In [160]:
postal_codes = pd.read_csv('./CanadianPostalCodes.csv')

In [162]:
# save only FSA codes
postal_codes.loc[:, ['FSA']].to_csv('./FSA.csv', index=False)

In [5]:
postal_codes.head(3)

,PostalCode,FSA,Latitude,Longitude,Place Name,FSA1,FSA-Province,AreaType
0,A0A0A0,A0A,48.567450,-54.843225,Gander,A,10.0,Rural
1,A0A1A0,A0A,47.007347,-52.958921,Aquaforte,A,10.0,Rural
2,A0A1B0,A0A,47.362280,-53.293993,Avondale,A,10.0,Rural


In [155]:
def deg_to_rad(x): # convert lat, lon pair from deg to radians
    print(x)
    return [x[0] * np.pi / 180, x[1] * np.pi / 180]

In [50]:
arr = np.asarray(postal_codes.loc[:, ['Latitude', 'Longitude']])
arr_rad = np.array([ deg_to_rad(x) for x in arr ])
point = [43.9048786, -79.4450157]
point_rad = np.array(deg_to_rad(point))

In [159]:
print(point_rad)

[ 0.76628469 -1.3865771 ]


In [41]:
def print_nearest(distances, fsa_codes):
    for dist, fsa in zip(distances, fsa_codes):
        print(f'{fsa} - {rad_to_km(dist):.4f}km')

### Naive Method

In [88]:
%%time
distances = np.sum((arr-point)**2,axis=1)
fsa_lookup = np.asarray(postal_codes.loc[:, ['FSA']])

p = distances.argsort(kind='mergesort')
distance_sorted = distances[p][:100]
nearest = fsa_lookup[p][:100]

Wall time: 169 ms


In [89]:
u, indices = np.unique(nearest, return_index=True)
i = sorted(indices)[:4] # pick 3 closest unique postal codes

print_nearest(distance_sorted[i], nearest[i])

['L4S'] - 0.0002km
['L4E'] - 0.0953km
['L4C'] - 0.2783km


### BallTree Method

In [138]:
%%time
tree = BallTree(arr_rad, leaf_size=2, metric='haversine') # initialize index

Wall time: 1.85 s


In [131]:
import pickle
pickle.dump(tree, open('ball_postal_index.p', "wb"))

In [84]:
%%time
dist, ind = tree.query([point_rad], k=100) 

Wall time: 2 ms


In [149]:
# find postal codes in a radius
%%time
radius = km_to_rad(5)
point_rad = np.array(deg_to_rad(point))
ind = tree.query_radius([point_rad], r=[radius]) 

Wall time: 1 ms


In [156]:
point_rad = np.array(deg_to_rad(point))

[43.9048786, -79.4450157]


In [154]:
fsa = np.asarray(postal_codes.loc[:, ['FSA']])
np.unique(fsa[ind[-1]])

array(['L0G', 'L3T', 'L4A', 'L4B', 'L4C', 'L4E', 'L4M', 'L4S', 'L6A',
       'L6B', 'M2M', 'M2N'], dtype=object)

In [85]:
nearest = np.asarray(postal_codes.loc[ind[-1], ['FSA']])

u, indices = np.unique(nearest, return_index=True)
i = sorted(indices)[:4] # pick 3 closest unique postal codes

print_nearest(dist[-1][i], nearest[i])

['L4S'] - 0.0171km
['L4E'] - 0.4244km
['L4C'] - 0.7263km


In [129]:
nearest[i].flatten()

array(['L4S', 'L4E', 'L4C'], dtype=object)

* Lookup is DAMN fast.... holy shit, building the tree takes some time as expected
* Huge advantage of being able to use haversine metric

In [43]:
# convert radians to km
def rad_to_km(rad):
    r = 6371 # earth radius in km
    return rad*r # radians = arc_len / radius

In [132]:
# convert radians to km
def km_to_rad(km):
    r = 6371 # earth radius in km
    return km/r # radians = arc_len / radius

### HNSWLIB

In [91]:
%%time
hp = hnswlib.Index(space = 'l2', dim = 2)
hp.init_index(max_elements = len(arr), ef_construction = 200, M = 16)
arr_labels = np.asarray(postal_codes.index) # need unique data labels
hp.add_items(arr, arr_labels)
# Controlling the recall by setting ef:
hp.set_ef(150) # ef should always be > k

Wall time: 8.28 s


In [107]:
%%time
# Query dataset, k - number of closest elements (returns 2 numpy arrays)
labels, distances = hp.knn_query([point], k = 100)

Wall time: 499 µs


In [111]:
fsa = np.asarray(postal_codes.loc[:, ['FSA']])

In [120]:
nearest = fsa[labels[-1]]

u, indices = np.unique(nearest, return_index=True)
i = sorted(indices)[:6] # pick 5 closest unique postal codes

print_nearest(distances[-1][i], nearest[i])

['L4S'] - 0.0002km
['L4E'] - 0.0953km
['L4C'] - 0.2780km


* index creation is slower than balltree
* lookup speed is ridiculous

In [96]:
# save
import pickle
pickle.dump(hp, open('postal_index.p', "wb"))

In [97]:
%%time
hp = pickle.load(open('postal_index.p', "rb"))
# Query dataset, k - number of closest elements (returns 2 numpy arrays)
labels, distances = hp.knn_query([point], k = 100)

Wall time: 404 ms


In [98]:
hp.save_index("postal_index.bin")

In [103]:
%%time
p = hnswlib.Index(space='l2',dim=2)
p.load_index('postal_index.bin')
# Query dataset, k - number of closest elements (returns 2 numpy arrays)
labels, distances = p.knn_query([point], k = 100)

Wall time: 1.19 s


In [122]:
a = np.array([9, 2, 4])